# Morbilidad Hospitalaria España 2016-2019
[Datos INE](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176778&menu=resultados&secc=1254736195291&idp=1254735573175#!tabs-1254736195291)

In [1]:
import pandas as pd
import datetime

In [2]:
morbi16 = pd.read_csv('../data/morbilidad/EMH_2016_G01.csv',sep='\t')
morbi17 = pd.read_csv('../data/morbilidad/EMH_2017_G01.csv',sep='\t')
morbi18 = pd.read_csv('../data/morbilidad/EMH_2018_G01.csv',sep='\t')
morbi19 = pd.read_csv('../data/morbilidad/EMH_2019_G01.csv',sep='\t')

In [4]:
morbilidad = pd.concat([morbi16,morbi17,morbi18,morbi19])

In [5]:
morbilidad.sample(5)

,Norden,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,EdadMes,EdadDias,DiasEstancia,Factor
811324,10603543,8,2,8,1,7112017,G3183,1,84,5,19,14,1.01095
653166,3189210,6,1,6,2,12092019,E118,1,42,6,26,16,1.00000
422043,39577874,44,2,44,1,8082018,C787,3,87,2,11,2,1.00000
716586,40924156,46,1,33,2,26072018,F200,2,44,6,0,8,2.05330
497224,38283428,41,2,41,1,9062016,D259,1,41,11,23,3,1.02081


In [6]:
morbilidad = morbilidad[morbilidad["DiagPrin"].str.contains('^F')]

In [7]:
morbilidad.DiagPrin = morbilidad.DiagPrin.str.strip()

In [8]:
icd10 = pd.read_csv('../data/icd10.csv')

In [9]:
icd10_psic = icd10[icd10["code"].str.contains('^F')]
#el código de las enferemades mentales empeizan por F

In [10]:
icd10_psic.sample(3) #importamos la tabla de correspondencia entre código de diagnóstico y nombre

,code,diag
4011,F1098,"Alcohol use, unspecified with alcohol-induced ..."
4472,F330,"Major depressive disorder, recurrent, mild"
4157,F1412,Cocaine abuse with intoxication with delirium


In [11]:
morbilidad.drop(['Norden','EdadDias','EdadMes'],axis=1,inplace = True)

In [12]:
morbilidad.DiagPrin.unique()[0:20] #extraer F + 2 primero dígitos para tener los tipos de enfermedasdes mentales agrupadas

array(['F0150', 'F0151', 'F0390', 'F0391', 'F04', 'F05', 'F060', 'F061',
       'F062', 'F0630', 'F0631', 'F0632', 'F0633', 'F0634', 'F064',
       'F068', 'F070', 'F0781', 'F0789', 'F079'], dtype=object)

In [13]:
morbilidad["DiagGen"] = morbilidad.DiagPrin.str.extract(r'(^F..)')

In [14]:
morbilidad.DiagGen.unique()

array(['F01', 'F03', 'F04', 'F05', 'F06', 'F07', 'F09', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44',
       'F45', 'F48', 'F50', 'F51', 'F52', 'F53', 'F55', 'F59', 'F60',
       'F63', 'F64', 'F65', 'F66', 'F68', 'F69', 'F70', 'F71', 'F72',
       'F73', 'F78', 'F79', 'F80', 'F81', 'F82', 'F84', 'F88', 'F89',
       'F90', 'F91', 'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

In [15]:
diagnos = pd.read_csv("../data/diagnos.csv")

In [16]:
diagnos.sample(5)  #importamos la tabla de correspondencia entre código de diagnóstico agrupadas y nombre

,code,diag
7,F70-79,Retraso mental
52,F50,Trastornos de la ingestión de alimentos
57,F55,Abuso de sustancias que no producen dependencia
13,F02,Demencia en otras enfermedades clasificadas
53,F51,Trastornos del sueño no orgánicos


In [17]:
dicc_diag = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_diag[r.code] = r.diag

In [18]:
morbilidad["FFF"] = morbilidad.DiagGen.replace(dicc_diag) #columna con subclasificación diagnoósticos.

In [19]:
#listas de diágnosticos para agrupar por tipo.
F0 = ["F00","F01","F02","F03","F04","F05","F06","F07","F08","F09"]
F1 = ["F10","F11","F12","F13","F14","F15","F16","F17","F18","F19"]
F2 = ["F20","F21","F22","F23","F24","F25","F26","F27","F28","F29"]
F3 = ["F30","F31","F32","F33","F34","F35","F36","F37","F38","F39"]
F4 = ["F40","F41","F42","F43","F44","F45","F46","F47","F48","F49"]
F5 = ["F50","F51","F52","F53","F54","F55","F56","F57","F58","F59"]
F6 = ["F60","F61","F62","F63","F64","F65","F66","F67","F68","F69"]
F7 = ["F70","F71","F72","F73","F74","F75","F76","F77","F78","F79"]
F8 = ["F80","F81","F82","F83","F84","F85","F86","F87","F88","F89"]
F9 = ["F90","F91","F92","F93","F94","F95","F96","F97","F98"]
F99 = "F99"

In [20]:
cabezera = {'F00-F09': 'Trastornos mentales orgánicos, incluidos los trastornos sintomáticos',
 'F10-F19': 'Trastornos mentales y de comportamiento debidos al consumo de psicotrópicos',
 'F20-29': 'Esquizofrenia, trastornos esquizotípicos y trastornos delirantes',
 'F30-39': 'Trastornos del estado de animo, trastornos afectivos',
 'F40-49': 'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos',
 'F50-59': 'Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos',
 'F60-69': 'Trastornos de la personalidad y del comportamiento en adultos',
 'F70-79': 'Retraso mental',
 'F80-89': 'Trastornos del desarrollo psicológico',
 'F90-F98': 'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez o en la adolescencia'}

In [21]:
capi = diagnos.head(11) #selección de los grupos de diagnóstico principales

In [22]:
capi

,code,diag
0,F00-F09,"Trastornos mentales orgánicos, incluidos los t..."
1,F10-F19,Trastornos mentales y de comportamiento debido...
2,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra..."
3,F30-39,"Trastornos del estado de animo, trastornos afe..."
4,F40-49,"Trastornos neuróticos, trastornos relacionados..."
5,F50-59,Síndromes del comportamiento asociados con alt...
6,F60-69,Trastornos de la personalidad y del comportami...
7,F70-79,Retraso mental
8,F80-89,Trastornos del desarrollo psicológico
9,F90-F98,Trastornos emocionales y del comportamiento qu...


In [23]:
#creamos un diccionario con los diagnósticos y sus nombres
cabezera = {}
for i,r in capi.iterrows():
    cabezera[r.code] = r.diag
cabezera

{'F00-F09': 'Trastornos mentales orgánicos, incluidos los trastornos sintomáticos',
 'F10-F19': 'Trastornos mentales y de comportamiento debidos al consumo de psicotrópicos',
 'F20-29': 'Esquizofrenia, trastornos esquizotípicos y trastornos delirantes',
 'F30-39': 'Trastornos del estado de animo, trastornos afectivos',
 'F40-49': 'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos',
 'F50-59': 'Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos',
 'F60-69': 'Trastornos de la personalidad y del comportamiento en adultos',
 'F70-79': 'Retraso mental',
 'F80-89': 'Trastornos del desarrollo psicológico',
 'F90-F98': 'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez o en la adolescencia',
 'F99': 'Trastornos mentales sin especificar'}

In [24]:
#creamos un diccionario con las correspondencias de los subtipos y tipos principales de diagnósticos
capitulando = {}
for f in F0:
    capitulando[f] = cabezera['F00-F09']
for f in F1:
    capitulando[f] = cabezera['F10-F19']
for f in F2:
    capitulando[f] = cabezera['F20-29']
for f in F3:
    capitulando[f] = cabezera['F30-39']
for f in F4:
    capitulando[f] = cabezera['F40-49']
for f in F5:
    capitulando[f] = cabezera['F50-59']
for f in F6:
    capitulando[f] = cabezera['F60-69']
for f in F7:
    capitulando[f] = cabezera['F70-79']
for f in F8:
    capitulando[f] = cabezera['F80-89']
for f in F9:
    capitulando[f] = cabezera['F90-F98']
capitulando[F99] = cabezera['F99']

In [25]:
morbilidad.sample(5)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,DiagGen,FFF
763559,41,2,41,2,26032019,F3110,1,24,20,1.00000,F31,Trastorno bipolar
792081,35,1,35,2,25092019,F4323,1,53,8,1.00000,F43,Reacción al stress grave y trastornos de adapt...
690663,8,1,8,2,13022018,F1010,1,41,10,1.09860,F10,Trastornos mentales y de comportamiento debido...
728318,36,2,36,2,18122017,F22,1,65,16,1.00000,F22,Trastornos delirantes persistentes
751442,8,1,8,1,10062019,F251,1,56,42,1.08056,F25,Trastornos esquizoafectivos


In [26]:
morbilidad["Fgen"] = morbilidad.DiagGen.map(capitulando) #creamos nueva columnas

In [28]:
discapacidad = morbilidad[morbilidad["Fgen"] == 'Retraso mental'].index #dropeamos el tipo de retraso mental
morbilidad.drop(discapacidad,inplace = True)

In [30]:
reverse_dic = {} #diccioanrio con nombre_diag como key y con código como value
for i,r in capi.iterrows():
    reverse_dic[r.diag] = r.code
reverse_dic

{'Trastornos mentales orgánicos, incluidos los trastornos sintomáticos': 'F00-F09',
 'Trastornos mentales y de comportamiento debidos al consumo de psicotrópicos': 'F10-F19',
 'Esquizofrenia, trastornos esquizotípicos y trastornos delirantes': 'F20-29',
 'Trastornos del estado de animo, trastornos afectivos': 'F30-39',
 'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos': 'F40-49',
 'Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos': 'F50-59',
 'Trastornos de la personalidad y del comportamiento en adultos': 'F60-69',
 'Retraso mental': 'F70-79',
 'Trastornos del desarrollo psicológico': 'F80-89',
 'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez o en la adolescencia': 'F90-F98',
 'Trastornos mentales sin especificar': 'F99'}

In [31]:
morbilidad["CodGen"] = morbilidad.Fgen.replace(reverse_dic)

In [34]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,DiagGen,FFF,Fgen,CodGen
738244,29,1,29,2,28122018,F29,1,24,6,1.01658,F29,Psicosis no orgánica sin especificar,"Esquizofrenia, trastornos esquizotípicos y tra...",F20-29
789472,14,2,14,2,23102017,F609,1,25,7,1.00000,F60,Trastorno de personalidad específico,Trastornos de la personalidad y del comportami...,F60-69
708585,28,1,28,1,15122019,F09,1,49,318,1.02096,F09,Trastornos mentales orgánicos o sintomáticos s...,"Trastornos mentales orgánicos, incluidos los t...",F00-F09


In [35]:
#creamos fecha a través de FxAlta
morbilidad.FxAlta = morbilidad.FxAlta.astype('string')

In [36]:
morbilidad['fecha_al'] = pd.to_datetime({'year': morbilidad.FxAlta.str[-4:],
                           'month': morbilidad.FxAlta.str[-6:-4],
                           'day': morbilidad.FxAlta.str[:-6]
                           })

In [37]:
#creamos fecha ingreso restando a fecha_al el número de día ingresado (DiasEstancia)
morbilidad['fecha_ing'] = morbilidad.apply(lambda fila: fila['fecha_al'] - datetime.timedelta(days = fila['DiasEstancia']), axis= 1)

In [38]:
#diccionarios códigos
dic_pro ={2:'Albacete',3:'Alicante',4:'Almería',1:'Álava',33:'Asturias',5:'Ávila',6:'Badajoz',7:'Balears, Illes',
8:'Barcelona',48:'Bizkaia',9:'Burgos',10:'Cáceres',11:'Cádiz',39:'Cantabria',12:'Castellón',13:'Ciudad Real',
14:'Córdoba',15:'Coruña, A',16:'Cuenca',20:'Gipuzkoa',17:'Girona',18:'Granada',19:'Guadalajara',21:'Huelva',22:'Huesca',
23:'Jaén',24:'León',25:'Lleida',27:'Lugo',28:'Madrid',29:'Málaga',30:'Murcia',31:'Navarra',32:'Ourense',34:'Palencia',
35:'Palmas, Las',36:'Pontevedra',26:'Rioja, La',37:'Salamanca',38:'Santa Cruz de Tenerife',40:'Segovia',41:'Sevilla',42:'Soria',
43:'Tarragona',44:'Teruel',45:'Toledo',46:'Valencia',47:'Valladolid',49:'Zamora',50:'Zaragoza',51:'Ceuta',
52:'Melilla',53:'Extranjeros'}
dic_sexo = {1:'Hombre',2:'Mujer'}
dic_entrada = {1:'Ordinario',2:'Urgente'}
dic_alta = {1: "Curación",2:'Traslado',3:'Fallecimiento',4:'Otras'}

In [39]:
#creamos nuevas columnas /cambiamos datos de columnas en función de los códigos
morbilidad['nomProvHosp'] = morbilidad.ProvHosp.map(dic_pro) #ProvResi
morbilidad['nomProvResi'] = morbilidad.ProvResi.map(dic_pro)
morbilidad['Entrada'] = morbilidad.DiagEntr.map(dic_entrada)
morbilidad['Alta'] = morbilidad.MotivoAlta.map(dic_alta)
morbilidad['SexNom'] = morbilidad.Sexo.map(dic_sexo)

In [41]:
morbilidad.drop(['FxAlta','DiagPrin'],axis=1,inplace = True)

In [43]:
morbilidad.columns

Index(['ProvHosp', 'Sexo', 'ProvResi', 'DiagEntr', 'MotivoAlta', 'EdadAnios',
       'DiasEstancia', 'Factor', 'DiagGen', 'FFF', 'Fgen', 'CodGen',
       'fecha_al', 'fecha_ing', 'nomProvHosp', 'nomProvResi', 'Entrada',
       'Alta', 'SexNom'],
      dtype='object')

In [44]:
morbilidad.columns = ['ProvHosp', 'Sexo', 'ProvResi', 'DiagEntr', 'MotivoAlta', 'Edad',
       'Dias', 'Factor', 'CodeDiag', 'Diag', 'DiagGen', 'CodeGen',
       'fecha_al', 'fecha_ing', 'nomProvHosp', 'nomProvResi', 'Entrada',
       'Alta', 'SexNom']

In [45]:
morbilidad = morbilidad.iloc[:,[1,18,5,0,14,2,15,3,16,13,12,6,4,17,8,9,11,10,7]]

In [46]:
morbilidad.sample(3)

,Sexo,SexNom,Edad,ProvHosp,nomProvHosp,ProvResi,nomProvResi,DiagEntr,Entrada,fecha_ing,fecha_al,Dias,MotivoAlta,Alta,CodeDiag,Diag,CodeGen,DiagGen,Factor
789369,1,Hombre,49,10,Cáceres,6,Badajoz,2,Urgente,2017-08-03,2017-08-14,11,1,Curación,F60,Trastorno de personalidad específico,F60-69,Trastornos de la personalidad y del comportami...,1.33304
705027,2,Mujer,32,24,León,24,León,2,Urgente,2016-03-25,2016-04-04,10,1,Curación,F23,Trastornos psicóticos agudos y transitorios,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra...",1.57046
762321,2,Mujer,39,33,Asturias,33,Asturias,1,Ordinario,2016-07-18,2016-08-05,18,1,Curación,F60,Trastorno de personalidad específico,F60-69,Trastornos de la personalidad y del comportami...,1.00000


In [47]:
morbilidad.to_csv("../data/morbilidad_Fgen.csv",index=False)